<font size=7 color=green>Summary: 1565/1600</font>

In [ ]:
import re 
import spacy  
from spacy import displacy 
import en_core_web_sm 
from pathlib import Path 
from IPython.display import display, Markdown, Latex
import sys 
import csv 
import pandas as pd 
import numpy as np 
import os 
from os import path 
from datetime import datetime 
from urllib.parse import urlparse, urljoin 
import requests 
import ntpath 
from bs4 import BeautifulSoup, Comment 
import time 
from pprint import pprint  
import shutil 
import tldextract 
import multiprocessing 
from multiprocessing import Process, Queue, cpu_count 
import os 
from os import path, listdir
###
from IPython.core.debugger import set_trace
import random
from fractions import Fraction
import nltk
import pprint as pp
from collections import Counter
from itertools import repeat, chain
from nltk.corpus import wordnet
from nltk.stem import PorterStemmer 

In [ ]:
nltk.download_shell()

In [ ]:
from IPython.display import display, Markdown, Image
display(Markdown(
"""
# Midterm Project Learning Status Check
## <font color="red">Due: Feb 1, 2020</font>
## Open Book Midterm:
<ol>
    <li><b>(200 Points)Project Statement</b><br>: What are we doing? and in high level, how we are doing it <br>You may review Phase I/II PowerPoints.<br>Words limit: 500 
    <br></li>
    <br><li><b>(200 Points)Impersonate and Search</b><br>:Sign in US News Compass and conduct 3 to 5 searches. <br> Save the results to HTML files (one file per search) 
    <br><a href="https://premium.usnews.com/best-colleges/myfit?gpa=4.0&mode=table&sat-composite=1580">Compass Search</a>
    <br>A sample file from search may look like this: <br>
    <a href='./img/OneSampleCompassHtml.html'> One Sample file</a>
    </li>
    <br><li><b>(800 Points)Python and Content</b><br>Design and Implement a Class to extrat result from pages generated 
    from previous question. <br>Ideal output format should reflect at the following information:  </li>
    <br> Search Criteria: <br><img src="./img/CompassSearchFields.PNG" width="500"/>
    <br> Search Results: <br><img src="./img/CompassSearchResults.PNG" width="500"/></li>
    <br><li><b>(400 Points)Scratching NLTK</b><br>Crawl the following page<br>
    <a href="https://www.nme.com/features/greta-thunberg-full-speech-to-the-un-2019-climate-change-summit-2550824">Greta Thunberg UN Speech</a></li>
    <br> Tasks: 
    <br><ul>
        <li>Get the content of her speech</li>
        <li>Build a dataset that list all words in this speech and how many times each word appears</li>
        <li>Sort the list of words by <br>a).Alphabetically. <br> b). by frequency
        <li>Lemmatize all words</li>
        <li>Choose ~30 "difficult" words and get the synonyms and antoyms for them.</li> 
    </ul>
    </li>
</ol>
"""
))   

<font size=6 color=olive>Part 1: 200/200</font>
<br>Good statement

In [ ]:
from IPython.display import display, Markdown, Image
display(Markdown( 
"""
# 1 Project Management
<body>
    <b>What are we doing? </b> 
    <br/>
    <p>
      What we are doing is to help the students to find the best colleges based on their info. 
    </p>
    <br/>  
    <b>How are we doing it?</b>
    <br/>
    <p>
      We provide a React application as UI that can be accessed in a web brower.<br/>
      Students can do two things in our app: 
      (1) students can enter their information like GPA, SAT/ACT score etc or other search criteria like
      desired school location or tuition range, and our application will give a list of recommended colleges.  
      (2) students can enter their information like GPA and their desired college, and our app will do a gap
      analysis to give suggestions of actions to be taken in order to get into that desired college.<br/>
      As mentioned the UI front end is using react and the backend is handled through node.js. The college 
      data will be gathered by web crawling, and AI and python is used to analyze the input and data 
      to generate output.     
   </P>  
</body>
"""
))

<font size=6 color=olive>Part 2/3: 400/400</font>
<br>Excellent

In [ ]:
#2. Check github for files.

In [ ]:
#3
class CollegeData():
    
    
    def __init__(self, htmlFile='C:\\Users\\kentt\\stempro\\hw\\midterms\\data\\college3.html'):
        htmlPath = Path(htmlFile) 
        self.criteriaDF = pd.DataFrame()
        self.resultDF = pd.DataFrame()
        self.resultFile = htmlPath.parent / (htmlPath.stem +".csv")
        self.htmlFile = htmlFile
    
    def run(self):
        soup=BeautifulSoup(open(self.htmlFile)) 
        self.getCriteria(soup) 
        self.getResult(soup)
        self.saveToFile()
    
    def getCriteria(self, soup):
   
    # get search criteria
    # search criteria is a dictionary like {'GPA': ' 3.5', 'SAT Composite': ' 1100', 'ACT': ' 22'', 'Tuition and Fees': ' < $0 - $26,000'}
        criteria={}
        for item in soup.find_all('button'): 
            for x in item.find_all("span"):
                query = x.text.strip()
                a = query.split(':') 
                criteria[a[0]]=a[1]
        print("My Fit College Search\n") 
        self.criteriaDF = pd.DataFrame(criteria.items(), columns=["Criteria", "Values"])
        print(self.criteriaDF.to_string(index=False))
       # criteriaDF.to_csv('file3search.csv',index=False)
        
    
    def getResult(self, soup):
    # get search result
    # search result is a list(table) with each element also a list(tr) and 
    # each element (td) of the inner list is each field
    # e.g [['87', 'California State University--San Bernardino', 'N/A', 'N/A', 'N/A', '3.3'], ['87', 'California State University--Stanislaus', '495', '507', '19', '3.3'], ['85', 'California State University--Fresno', '506', '518', '19', '3.5']]
        result = []
        for item1 in soup.find_all('table'):
            for tr in item1.find_all("tr"):
                row =[]
                for cell in tr.find_all(["th","td"]):
                    if cell.name == "th":
                        row.append(cell.text.strip())
                    elif cell.name == "td":
                        for div in cell.find_all("div", {"class":"show-for-medium-up"}):
                            texts = div.find_all(text=True)
                            text2 = filter(lambda text: text.strip(), texts)#remove empty string or \n
                            validTexts = list(map(lambda x: re.sub(r'[\n ] ','',x), text2))#
                            row.append(validTexts[0])
                result.append(row)
            
        headers = result.pop(0)
        self.resultDF = pd.DataFrame(result, columns=headers)
        print(self.resultDF.to_string(index=False))
        #resultDF.to_csv('file3.csv',index=False)
    
    
    def saveToFile(self):
        #first write criteria, then write the search result
        self.criteriaDF.to_csv(self.resultFile, index=False, mode='w')
        self.resultDF.to_csv(self.resultFile, index=False, mode='a')
        
files = ['C:\\Users\\kentt\\stempro\\hw\\midterms\\data\\college3.html','C:\\Users\\kentt\\stempro\\hw\\midterms\\data\\college2.html','C:\\Users\\kentt\\stempro\\hw\\midterms\\data\\college1.html']        
for file in files: 
    cd=CollegeData(file)
    cd.run()

<font size=6 color=olive>Part 4: 200/200</font>
<br>Excellent

In [ ]:
#4

class nltkSpeech():
    
    def __init__(self, url="https://www.nme.com/features/greta-thunberg-full-speech-to-the-un-2019-climate-change-summit-2550824"):
        self.speech = []
        self.url = url
        

    def getSpeech(self):
        res = requests.get(self.url)
        htmlPage = res.content
        soup = BeautifulSoup(htmlPage, 'lxml')
        text= []
        dis = []
        firstp = soup.p    
        for x in firstp.find_all_next('p'):
            sentence = x.text.strip()
           # set_trace()
            if not (sentence.startswith("Everyone") or sentence.startswith("NME") or sentence.startswith('©')):   
                self.speech.extend(re.sub(r'[.,()“”!?]','',sentence).lower().split())  
        
    
    def freSort(self):
        print("List of words sorted by frequency")
        print("*************************************************")
        counts = Counter(self.speech)
        pp.pprint(counts)
        print('\n')
    
    def alphaSort(self):
        print("List of words sorted alphabetically")#Q: should duplidate words be listed just once?
        print("*************************************************")
        pp.pprint(sorted(self.speech))
        print('\n')
        
   
    
    def lemmatizing(self):
        print('Lemmatizing all words with WordNetLemmatizer\n')
        print("*************************************************")
        print('Word\t\t\t\tLemma')
        print('-------------------------------------------------')
        lm = nltk.stem.WordNetLemmatizer()
        
        for e in self.speech:
            lol=int(len(e)/8)
            if lol == 1:
                print(e+'\t\t\t'+lm.lemmatize(e))
            elif lol == 2:
                print(e+'\t\t'+lm.lemmatize(e))
            else:
                print(e+'\t\t\t\t'+lm.lemmatize(e))         
            
        print('\n')    
            
    def syn(self):
        words = ['urgency','toxic','technologies','sucking','situation','setting','presented','plea','nowhere','odds','matter','leaders','justice','kept','irreversible','impassioned','heartfelt','gigatons','extinction','equity','eternal','crystal','consequences','childhood','betrayal','billions','aspects','additional','activist']
        print("List of synonyms and antonyms for 30 words")
        print("*************************************************")
        for word in words:
            synonyms = []
            antonyms = []
            for sqyn in wordnet.synsets(word):
                for lm in sqyn.lemmas():
                    synonyms.append(lm.name())
                    if lm.antonyms():
                        antonyms.append(lm.antonyms()[0].name())
            pp.pprint('Word: ' + str(word))
            pp.pprint('Synonyms:' + str(synonyms))
            pp.pprint('Antonyms:' + str(antonyms))
            
            print('\n')
    
    def run(self):
        self.getSpeech()
        self.freSort()
        self.alphaSort()
        self.lemmatizing()
        self.syn()

ntlkspeech=nltkSpeech()
ntlkspeech.run()

    
    

In [2]:
display(Markdown(
"""
# Python Exam (For Python students):
<ol>
<li>(100 points)Write a program that contains a while loop that repeatedly asks the user to enter numeric miles they run each week.  Append each run to a list called “distance”. You may use this format [(“week1”, 23.1), (“week2”, 12.2)…]  
</li><br><li>(100 points)Write a program that get the max, min, average, number of runs etc. 
</li><br><li>(100 points)Write a program that prompts the user to enter in an unlimited number of colors. Test to make sure that the user did not already enter a color before storing it (i.e. “We’re sorry, but you’ve already added the color ‘yellow’”). When the user types the string “exit” you should stop collecting colors. The print out the colors entered in alphabetical order.  
</li><br><li>(100 points)Write a function that generates a random letter from the string ‘BINGO’. Here is a sample running of this program: x = generate_bingo_letter() print (x) 
</li><br><li>(100 points)Define a function fraction(fstring), where the argument passed in stores a fraction as a string. The function should return a floating point number that is the value of the fraction. 
 
Examples of using the function: <br>
         print(fraction(“3/4”))  # prints  .750 <br>
         print(fraction(“1/3”))  # prints   0.333 <br>
     
 
</li><br><li>(Bonus. 300 Points) Write a function balance_parens(astring) that inputs a string with parenthesized sub-­‐strings, which may be nested (e.g., could be a Python arithmetic expression). The function returns True if all of the parentheses that are opened are properly closed, otherwise it returns False. 
<br>
For example, the following string is balanced: 
<br>
“ A * ( (B + C) – (D – E) )” 
<br>
And the following string is not balanced: 
<br> 
“  A ((B * C)  + (D      
</li>
</ol>

"""
))


# Python Exam (For Python students):
<ol>
<li>(100 points)Write a program that contains a while loop that repeatedly asks the user to enter numeric miles they run each week.  Append each run to a list called “distance”. You may use this format [(“week1”, 23.1), (“week2”, 12.2)…]  
</li><br><li>(100 points)Write a program that get the max, min, average, number of runs etc. 
</li><br><li>(100 points)Write a program that prompts the user to enter in an unlimited number of colors. Test to make sure that the user did not already enter a color before storing it (i.e. “We’re sorry, but you’ve already added the color ‘yellow’”). When the user types the string “exit” you should stop collecting colors. The print out the colors entered in alphabetical order.  
</li><br><li>(100 points)Write a function that generates a random letter from the string ‘BINGO’. Here is a sample running of this program: x = generate_bingo_letter() print (x) 
</li><br><li>(100 points)Define a function fraction(fstring), where the argument passed in stores a fraction as a string. The function should return a floating point number that is the value of the fraction. 
 
Examples of using the function: <br>
         print(fraction(“3/4”))  # prints  .750 <br>
         print(fraction(“1/3”))  # prints   0.333 <br>
     
 
</li><br><li>(Bonus. 300 Points) Write a function balance_parens(astring) that inputs a string with parenthesized sub-­‐strings, which may be nested (e.g., could be a Python arithmetic expression). The function returns True if all of the parentheses that are opened are properly closed, otherwise it returns False. 
<br>
For example, the following string is balanced: 
<br>
“ A * ( (B + C) – (D – E) )” 
<br>
And the following string is not balanced: 
<br> 
“  A ((B * C)  + (D      
</li>
</ol>



In [ ]:
display(Markdown(
"""
## You may use the following libraries for the exams. 
import re <br>
import spacy  <br>
from spacy import displacy <br>
import en_core_web_sm <br>
from pathlib import Path <br>
from IPython.display import display, Markdown, Latex  <br>
import sys <br>
import csv <br>
import pandas as pd <br>
import numpy as np <br>
import os <br>
from os import path <br>
from datetime import datetime <br>
from urllib.parse import urlparse, urljoin <br>
import requests <br>
import ntpath <br>
from bs4 import BeautifulSoup, Comment <br>
import time <br>
from pprint import pprint  <br>
import shutil <br>
import tldextract <br>
import multiprocessing <br>
from multiprocessing import Process, Queue, cpu_count <br>
import os <br>
from os import path, listdir  <br>
"""
))


<font size=6 color=olive>Part 5: Python test 765/800</font>

In [6]:
#1
def run(weeks=10):
    distance = []
    week = 1
    while week <= weeks:
        try:
            miles = input('How many miles have you run in week' + str(week) + '\n')
            milesint = float(miles)
            if milesint < 0:
                print("Please enter a positive number")
                continue
            pair = ('week' + str(week), milesint)
            week += 1
            distance.append(pair)
        except ValueError:
            print("Please enter a number")
            continue         
            
    return(distance)
run()

How many miles have you run in week1
-2
Please enter a positive number
How many miles have you run in week1
-3
Please enter a positive number
How many miles have you run in week1
4
How many miles have you run in week2
3
How many miles have you run in week3
2
How many miles have you run in week4
4
How many miles have you run in week5
-635
Please enter a positive number
How many miles have you run in week5
4
How many miles have you run in week6
4
How many miles have you run in week7
4
How many miles have you run in week8
4
How many miles have you run in week9
5
How many miles have you run in week10
5


[('week1', 4.0),
 ('week2', 3.0),
 ('week3', 2.0),
 ('week4', 4.0),
 ('week5', 4.0),
 ('week6', 4.0),
 ('week7', 4.0),
 ('week8', 4.0),
 ('week9', 5.0),
 ('week10', 5.0)]

###### <font size=6 color=green> Q1 95/100</font><div class="alert alert-info"> 
Comments: 
1. Nicely done
2. what about negative miles?
</div>

In [ ]:
#2
y=0
data= [('week1', 23.0),
 ('week2', 4334.0),
 ('week3', 324.0),
 ('week4', 4.0),
 ('week5', 35321.0),
 ('week6', 562134.0),
 ('week7', 43214.0),
 ('week8', 5.0),
 ('week9', 2354.0),
 ('week10', 5.0)]

number_of_runs = len(data)

max_miles=max(data, key=lambda x: x[1])
min_miles=min(data, key=lambda x: x[1])
avg=sum(x[1] for x in data)/number_of_runs

print("The farthest distance you've run is " + str(max_miles[1]) + ' miles ' + 'on ' + str(max_miles[0]))
print("The shortest distance you've run is " + str(min_miles[1]) + ' miles ' + 'on ' + str(min_miles[0]))
print("Your average distance ran is " + str(avg) + " miles for " + str(number_of_runs))
print("You've gone on " + str(number_of_runs) + " runs so far.")

        



<font size=6 color=green> Q2 100/100</font><div class="alert alert-info"> 
Comments: 
1. Well Done
2. it would be ideal if use result from Question 1
</div>

In [18]:
#3
def listcolors():
    colors = {}
    while True:
        usercolor = input("Add a color: ")
        usercolorhash = hash(usercolor)
        if usercolor == "exit":
            break
            
        if usercolorhash in colors:
            print("We’re sorry, but you’ve already added the color " + usercolor)
            continue
        else:
            colors[usercolorhash] = usercolor
            for key, value in sorted(colors.items(), key=lambda item: item[1]):
                print("%s" % (value))

            
listcolors()

Add a color: d
d
Add a color: d
We’re sorry, but you’ve already added the color d
Add a color: w
d
w
Add a color: e
d
e
w
Add a color: r
d
e
r
w
Add a color: r
We’re sorry, but you’ve already added the color r
Add a color: t
d
e
r
t
w
Add a color: d
We’re sorry, but you’ve already added the color d
Add a color: a
a
d
e
r
t
w
Add a color: v
a
d
e
r
t
v
w
Add a color: c
a
c
d
e
r
t
v
w
Add a color: s
a
c
d
e
r
s
t
v
w
Add a color: exit


In [ ]:
#3 use dictionary, hash the color as the key


<font size=6 color=green> Q3 90/100</font><div class="alert alert-info"> 
Comments: 
1. Well Done
2. Try dictionary. which will be O(1) instead of O(n)
</div>

In [ ]:
#4
def generate_bingo_letter():
    bingo="BINGO"
    
    rand = random.randint(0,4)        
    return bingo[rand]
    
    #return random.choice(bingo)


x = generate_bingo_letter()
print(x) 

<font size=6 color=green> Q4 100/100</font><div class="alert alert-info"> 
Comments: 
Well Done 
</div>

In [9]:
#5
def fraction(fstring):
    sum = 0
    splitfract=fstring.split()
    if len(splitfract) != 1 and len(splitfract) != 2:
        print("invalid fraction")
        sys.exit()
        
       
    for x in splitfract:
        try:
            a = Fraction(x)
            rounded = round(float(a),3)
            sum = sum + rounded
        except ValueError as e :
            print(e)
    return(sum)


fraction('74398231 3841234/1324132')
fraction('-1/100')
fraction('string')

Invalid literal for Fraction: 'string'


0

<font size=6 color=green> Q5 80/100</font><div class="alert alert-info"> 
Comments: 
<br>Solution not functioning. 
<br>Good thinking process 
<br>The questions assumes: the input is valid fraction. To validate a fration, you may consider using regular expression
</div>

In [ ]:
import re 
def fraction(fstring): 
    if re.match(r'^-?\d+\s\d+/\d+', fstr) or re.match(r'^-?\d*/\d+', fstr):

        try:
            parts=re.split(' |/',fstring.strip())
            is_neg=False
            num=0
            num=int(parts[0].strip())
            if num<0:
                is_neg=True
            if  len(parts)==3:
                num=abs(num)+float(int(parts[-2].strip())/int(parts[-1].strip()))
            elif len(parts)==2:
                num = float(abs(num)/int(parts[-1].strip()))
            else:
                num=abs(num)
            num=-num if is_neg==True else num
            print(f'Answer is: {num:.3f}')
        except Exception as e:
            print('error:',e)
    else:
        print('improper fraction format.')
     

In [ ]:
#6
def balance_parens(astring):
    thestack = []
    index = 0 
    while index < len(astring):
        lpar = astring[index]
        if lpar == "(":
            thestack.append(lpar)
        elif lpar == ')':
            if len(thestack) == 0:
                return False
            else:
                thestack.pop()
        index = index + 1 
    if len(thestack) == 0:
        return True
    else:
        return False


balance_parens("((( kdkdksadjiajsd))))")
        

<font size=6 color=green> Q6 300/300</font><div class="alert alert-info"> 
Comments: 
<br>good solution
<br>challenge: how about there are other types? such as: [ { ] }?
</div>